# HDT4 PREDICCIÓN

In [33]:
import nltk
from nltk import lm
from nltk.corpus import stopwords
from nltk.book import FreqDist
from nltk.util import ngrams
from nltk import word_tokenize, sent_tokenize
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE
from nltk.lm.models import Laplace
import matplotlib.pyplot as plt
from itertools import chain

In [3]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [2]:
tweets = []
blogs = []
news = []

# tweets
with open('./CleanData/en_US.twitter.txt.txt', 'r') as f:
    tweets = f.read()

with open('./CleanData/twitter.txt', 'w') as f:
    f.write(tweets)
    
with open('./CleanData/twitter.txt', 'r') as f:
    tweets = f.read()
    
# blogs
with open('./CleanData/en_US.blogs.txt.txt', 'r') as f:
    blogs = f.read()

with open('./CleanData/blogs.txt', 'w') as f:
    f.write(blogs)
    
with open('./CleanData/blogs.txt', 'r') as f:
    blogs = f.read()
    
# news
with open('./CleanData/en_US.news.txt.txt', 'r') as f:
    news = f.read()

with open('./CleanData/news.txt', 'w') as f:
    f.write(news)
    
with open('./CleanData/news.txt', 'r') as f:
    news = f.read()

In [3]:
print(len(tweets))
print(len(blogs))
print(len(news))

7744557
10036716
742861


- Juntar los 3 textos para entrenar los distintos modelos.

In [4]:
all_texts = tweets + blogs + news
print(len(all_texts))

18524134


### N Gramas

In [5]:
words = word_tokenize(all_texts)
tweets_1_gram = list(ngrams(words, 1))
tweets_2_gram = list(ngrams(words, 2))
tweets_3_gram = list(ngrams(words, 3))

In [6]:
tweets_1_gram[:5]

[('punk',), ('i',), ('unfollowed',), ('kanye',), ('because',)]

In [7]:
tweets_2_gram[:5]

[('punk', 'i'),
 ('i', 'unfollowed'),
 ('unfollowed', 'kanye'),
 ('kanye', 'because'),
 ('because', 'of')]

In [8]:
tweets_3_gram[:5]

[('punk', 'i', 'unfollowed'),
 ('i', 'unfollowed', 'kanye'),
 ('unfollowed', 'kanye', 'because'),
 ('kanye', 'because', 'of'),
 ('because', 'of', 'that')]

#### Tokenizacion de texto

In [9]:
tokenized_text = [list(map(str.lower, word_tokenize(sent))) for sent in sent_tokenize(all_texts)]

## Modelos de predicción

### MLE

#### Entrenamiento

In [10]:
train, vocab = padded_everygram_pipeline(4, tokenized_text)

In [11]:
mle = MLE(4)

In [12]:
mle.fit(train, vocab)
print(mle.vocab)

<Vocabulary with cutoff=1 unk_label='<UNK>' and 114388 items>


### Kneser-Ney

In [13]:
from nltk.lm.models import KneserNeyInterpolated
kneser = KneserNeyInterpolated(4)

In [14]:
kn_train, kn_vocab = padded_everygram_pipeline(4, tokenized_text)
kneser.fit(kn_train, kn_vocab)
print(kneser.vocab)

<Vocabulary with cutoff=1 unk_label='<UNK>' and 114388 items>


In [15]:
kneser.score('you', 'how are'.split())

0.7907372155097071

### Laplace

In [34]:
lp_train, lp_vocab = padded_everygram_pipeline(4, tokenized_text)
laplace = Laplace(4)
laplace.fit(lp_train, lp_vocab)
laplace.score('you', 'how are'.split())

0.0016227251313011464

## Funcion de posibles 3 palabras

### MLE

In [16]:
frase = input("Ingrese la frase: ")
predictor = mle.generate(3, frase.split())

Ingrese la frase: i am


In [17]:
for x in predictor:
    print(x,mle.score(x, frase.split())) 

so 0.05670391061452514
sweaty 0.0
from 0.0025139664804469273


### Kneser-Ney

In [18]:
frase = input("Ingrese la frase: ")
predictor_kn = kneser.generate(3, frase.split())

Ingrese la frase: i am


In [19]:
for x in predictor_kn:
    print(x, kneser.score(x, frase.split())) 

not 0.06994577545527271
promising 4.693140763017691e-09
weekly 4.693140763017691e-09


In [35]:
frase = input("Ingrese la frase: ")
predictor_lp = laplace.generate(3, frase.split())

Ingrese la frase: i really want


In [36]:
for x in predictor_lp:
    print(x, laplace.score(x, frase.split()))

to 0.00033201981634061736
avoid 8.737363587910984e-06
it 3.4949454351643935e-05
